In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import KFold

from tqdm import tqdm

import load_script
from params import *
from custom_layers import *
from training import *

In [2]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [3]:
train_dataset,train_target=load_script.load_dataset_1000hz(train=True)

In [4]:
test_dataset,test_target=load_script.load_dataset_1000hz(train=False)

In [5]:
train_dataset,train_target=load_script.load_dataset(train=True)

In [6]:
test_dataset,test_target=load_script.load_dataset(train=False)

# Normalization

In [7]:
train_dataset.shape

torch.Size([316, 28, 50])

In [8]:
test_dataset.shape

torch.Size([100, 28, 50])

In [9]:
mean=train_dataset.mean(0).view(1,28,50)
std=train_dataset.std(0).view(1,28,50)

In [10]:
train_dataset=(train_dataset-mean)/std
test_dataset=(test_dataset-mean)/std

# Defining the network

### Simple Network

In [ ]:
class ThreeLayers(nn.Module):
    
    num_my_conv_layers=3
    num_linear_layers=2
    
    def __init__(self,params):
        super(ThreeLayers,self).__init__()
        
        self.params=params
        
        layers=[]
        for i in range(self.num_my_conv_layers): 
            layers+=MyConv1D(*self.params[i]).layers
        
        layers.append(Flatten())
        layers.append(nn.Linear(*self.params[self.num_my_conv_layers]))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(0.6))
        
        layers.append(nn.Linear(*self.params[self.num_my_conv_layers+1]))
        
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

In [11]:
class FullConnect(nn.Module):
    num_my_conv_layers=0
    num_linear_layers=5
    
    def __init__(self,params):
        super(FullConnect,self).__init__()
        self.dropouts=[0.8,0.8,0.8,0.8, 0]
        self.params=params
        layers=[]
        layers.append(Flatten())
        for i in range(self.num_linear_layers):
            layers.append(nn.Linear(*self.params[self.num_my_conv_layers+i]))
            layers.append(nn.BatchNorm1d(self.params[self.num_my_conv_layers+i][1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.dropouts[i]))
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x
        

In [12]:
class FullConv(nn.Module):
    num_my_conv_layers=4
    num_linear_layers=0
    def __init__(self,params):
        super(FullConv,self).__init__()
        
        self.params=params
        layers=[]
        for i in range(self.num_my_conv_layers):
            layers+=MyConv1D(*self.params[i]).layers
        
        layers.append(Flatten())
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

# Training

## Full Convolutional training

In [ ]:
cv=CrossValidation(k=2,train_dataset=train_dataset,test_dataset=test_dataset,
                   train_target=train_target,test_target=test_target)

net_type=FullConv
optimizer_type=optim.Adam
criterion_type=nn.CrossEntropyLoss
network_params=NetworkParams(conv_filters=[28,14,7,2],conv_kernels=[5,9,11,20],dropout_rate=[0.8,0.8,0,0],\
                             stride=1,dilation=[3,1,1,1])
#network_params=NetworkParams(linear_filters=[244,2],dropout_rate=0.8,dilation=[5,1,1])
optimizer_params=OptimizerParams()
train_params=TrainParams(max_epoch=500,mini_batch_size=2*79)


params=Params(net_type,optimizer_type,criterion_type,network_params=network_params,\
              optimizer_params=optimizer_params,train_params=train_params)

cv(params)

# Train full connnected

In [13]:
cv=CrossValidation(k=2,train_dataset=train_dataset,test_dataset=test_dataset,
                   train_target=train_target,test_target=test_target,cuda=True)

net_type=FullConnect
optimizer_type=optim.Adam
criterion_type=nn.CrossEntropyLoss
#network_params=NetworkParams(conv_filters=[28,14,7,2],conv_kernels=[5,9,11,20],dropout_rate=[0.6,0.6,0,0],\
 #                            stride=1,dilation=[3,1,1,1])
network_params=NetworkParams(linear_filters=[500,100,50,2],conv1D=False)
optimizer_params=OptimizerParams()
train_params=TrainParams(max_epoch=1000,mini_batch_size=79)

params=Params(net_type,optimizer_type,criterion_type,network_params=network_params,\
              optimizer_params=optimizer_params,train_params=train_params,cuda=True)

cv(params, cross_validation=False, repetitions=1, repetitions_test=4)

TypeError: torch.FloatTensor constructor received an invalid combination of arguments - got (bool, int), but expected one of:
 * no arguments
 * (int ...)
      didn't match because some of the arguments have invalid types: ([31;1mbool[0m, [31;1mint[0m)
 * (torch.FloatTensor viewed_tensor)
 * (torch.Size size)
 * (torch.FloatStorage data)
 * (Sequence data)


In [ ]:
? nn.Conv1d

In [ ]:
list_diego=[1,2,3]
list_rosa=[4,5,6]
fiori=zip(list_diego,list_rosa)
for i in fiori: print(i)

In [ ]:
? zip

In [ ]:
? nn.Conv2d